In [ ]:
from IPython import get_ipython

ipython = get_ipython()
# Code to automatically update the HookedTransformer code as its edited without restarting the kernel
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/var/folders/pf/g3nr86yn4j71vzzmv6knwdhr0000gp/T/ipykernel_2118/4115428919.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/var/folders/pf/g3nr86yn4j71vzzmv6knwdhr0000gp/T/ipykernel_2118/4115428919.py:6: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [ ]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [ ]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

### Indirect Object Identification


In [ ]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [ ]:
example_prompt = "After John and Mary went to the store, John gave a bottle of milk to"

In [ ]:
example_answer = " Mary"

In [ ]:
prompt_format = [
    "When John and Mary went to the shops,{} gave the bag to",
    "When Tom and James went to the park,{} gave the ball to",
    "When Dan and Sid went to the shops,{} gave an apple to",
    "After Martin and Amy went to the park,{} gave a drink to",
]
names = [
    (" Mary", " John"),
    (" Tom", " James"),
    (" Dan", " Sid"),
    (" Martin", " Amy"),
]
# List of prompts
prompts = []
# List of answers, in the format (correct, incorrect)
answers = []
# List of the token (ie an integer) corresponding to each answer, in the format (correct_token, incorrect_token)
answer_tokens = []
for i in range(len(prompt_format)):
    for j in range(2):
        answers.append((names[i][j], names[i][1 - j]))
        answer_tokens.append(
            (
                model.to_single_token(answers[-1][0]),
                model.to_single_token(answers[-1][1]),
            )
        )
        # Insert the *incorrect* answer to the prompt, making the correct answer the indirect object.
        prompts.append(prompt_format[i].format(answers[-1][1]))
answer_tokens = torch.tensor(answer_tokens)
print(prompts)
print(answers)

['When John and Mary went to the shops, John gave the bag to', 'When John and Mary went to the shops, Mary gave the bag to', 'When Tom and James went to the park, James gave the ball to', 'When Tom and James went to the park, Tom gave the ball to', 'When Dan and Sid went to the shops, Sid gave an apple to', 'When Dan and Sid went to the shops, Dan gave an apple to', 'After Martin and Amy went to the park, Amy gave a drink to', 'After Martin and Amy went to the park, Martin gave a drink to']
[(' Mary', ' John'), (' John', ' Mary'), (' Tom', ' James'), (' James', ' Tom'), (' Dan', ' Sid'), (' Sid', ' Dan'), (' Martin', ' Amy'), (' Amy', ' Martin')]


In [ ]:
tokens = model.to_tokens(prompts, prepend_bos=True)

In [ ]:
tokens

tensor([[50256,  2215,  1757,   290,  5335,  1816,   284,   262, 12437,    11,
          1757,  2921,   262,  6131,   284],
        [50256,  2215,  1757,   290,  5335,  1816,   284,   262, 12437,    11,
          5335,  2921,   262,  6131,   284],
        [50256,  2215,  4186,   290,  3700,  1816,   284,   262,  3952,    11,
          3700,  2921,   262,  2613,   284],
        [50256,  2215,  4186,   290,  3700,  1816,   284,   262,  3952,    11,
          4186,  2921,   262,  2613,   284],
        [50256,  2215,  6035,   290, 15686,  1816,   284,   262, 12437,    11,
         15686,  2921,   281, 17180,   284],
        [50256,  2215,  6035,   290, 15686,  1816,   284,   262, 12437,    11,
          6035,  2921,   281, 17180,   284],
        [50256,  3260,  5780,   290, 14235,  1816,   284,   262,  3952,    11,
         14235,  2921,   257,  4144,   284],
        [50256,  3260,  5780,   290, 14235,  1816,   284,   262,  3952,    11,
          5780,  2921,   257,  4144,   284]])

In [ ]:
original_logits, cache = model.run_with_cache(tokens)

### Direct Logit Attribution

##### Flashcard 1

In [ ]:
W_U = model.W_U

In [ ]:
tokens = torch.tensor([69, 420])

In [ ]:
tokens

tensor([ 69, 420])

In [ ]:
W_U.shape

torch.Size([768, 50257])

Give `W_U` is the unembedding matrix

Extract the residual directions of `tokens`

In [ ]:
residual_directions = W_U[:, tokens]

In [ ]:
residual_directions.shape

torch.Size([768, 2])

##### Example 2

In [ ]:
correct_token = torch.tensor(5335)

In [ ]:
incorrect_token = torch.tensor(1757)

In [ ]:
type(model)

transformer_lens.HookedTransformer.HookedTransformer

In [ ]:
correct_token, incorrect_token

(tensor(5335), tensor(1757))

Compute the logit difference of the residual directions of two tokens above

In [ ]:
correct_residual_direction = model.tokens_to_residual_directions(correct_token)

In [ ]:
correct_residual_direction.shape

torch.Size([768])

In [ ]:
incorrect_residual_direction = model.tokens_to_residual_directions(incorrect_token)

In [ ]:
incorrect_residual_direction.shape

torch.Size([768])

In [ ]:
logit_diff_direction = correct_residual_direction - incorrect_residual_direction

In [ ]:
logit_diff_direction.shape

torch.Size([768])

##### Example 3

In [ ]:
unembedding = model.W_U

`768` is the dimension of an embedding vector, `50257` is the number of vocabulary

In [ ]:
unembedding.shape

torch.Size([768, 50257])

In [ ]:
correct_token, incorrect_token

(tensor(5335), tensor(1757))

Compute the logit difference of the residual directions of two tokens above

In [ ]:
correct_residual_direction = unembedding[:, correct_token]

In [ ]:
correct_residual_direction.shape

torch.Size([768])

In [ ]:
incorrect_residual_direction = unembedding[:, incorrect_token]

In [ ]:
incorrect_residual_direction.shape

torch.Size([768])

In [ ]:
logit_diff_direction = correct_residual_direction - incorrect_residual_direction

In [ ]:
logit_diff_direction.shape

torch.Size([768])

##### Example 4

In [ ]:
tokens = model.to_tokens(prompts, prepend_bos=True)

In [ ]:
original_logits, cache = model.run_with_cache(tokens)

In [ ]:
cache.compute_head_results()

In [ ]:
import torch
from einops import rearrange

In [ ]:
type(cache)

transformer_lens.ActivationCache.ActivationCache

In [ ]:
[key for key in cache.keys() if key.startswith("blocks.0.attn")]

['blocks.0.attn.hook_q',
 'blocks.0.attn.hook_k',
 'blocks.0.attn.hook_v',
 'blocks.0.attn.hook_attn_scores',
 'blocks.0.attn.hook_pattern',
 'blocks.0.attn.hook_z',
 'blocks.0.attn.hook_result']

Stack the output of all attention layer from layer `0` to layer `2` as bellow. Explain

In [ ]:
outputs = []

In [ ]:
for layer in range(3):
    outputs.append(cache[f"blocks.{layer}.attn.hook_result"])

The `outputs` list contains the attention outputs from three layers. The output shape for the first layer is `[8, 15, 12, 768]`, which corresponds to a batch size of 8, a sequence length of 15, 12 attention heads, and an embedding of 768.

In [ ]:
len(outputs)

3

In [ ]:
outputs[0].shape

torch.Size([8, 15, 12, 768])

The goal is to combine all the attention heads from the different layers, which are located along dimension 2 (or the last two dimensions). To do this, the `outputs` are concatenated along dimension 2.

In [ ]:
outputs = torch.cat(outputs, dim=-2)

After concatenation, the resulting output shape is `[8, 15, 36, 768]`, which corresponds to a batch size of 8, a sequence length of 15, 36 attention heads (combined from all layers).

In [ ]:
outputs.shape

torch.Size([8, 15, 36, 768])

##### Example 5

In [ ]:
outputs = rearrange(outputs, "... concat_head_index d_model -> concat_head_index ... d_model")

In [ ]:
outputs.shape

torch.Size([36, 8, 15, 768])

##### Example 6

In [ ]:
type(cache)

transformer_lens.ActivationCache.ActivationCache

Extract the residual direction of all heads from layer `0` to layer `2` using `transformer_lens`

In [ ]:
per_head_residual = cache.stack_head_results(layer=2)

In [ ]:
per_head_residual.shape

torch.Size([24, 8, 15, 768])

##### Example 6.1

In [ ]:
x = torch.arange(0, 8).view(2, 4)
y = torch.arange(0, 8).view(2, 4)

In [ ]:
x.shape, y.shape

(torch.Size([2, 4]), torch.Size([2, 4]))

`x` and `y` have batch size `2`, and dimension `4`

In [ ]:
x

tensor([[0, 1, 2, 3],
        [4, 5, 6, 7]])

In [ ]:
y

tensor([[0, 1, 2, 3],
        [4, 5, 6, 7]])

In [ ]:
output = (x * y).sum()

In [ ]:
output

tensor(140)

Perform a single operation on x and y using `einops` to do element-wise multiplication between them, and then sum up the resulting value

In [ ]:
from einops import einsum

In [ ]:
einops_output = einsum(x, y, "batch dim, batch dim ->")

In [ ]:
einops_output

tensor(140)

In [ ]:
output == einops_output

tensor(True)

##### Example 6.2

In [ ]:
_logits, _cache = model.run_with_cache(tokens)

In [ ]:
type(model)

transformer_lens.HookedTransformer.HookedTransformer

In [ ]:
_logits.shape

torch.Size([8, 15, 50257])

Manually calculate the logits of `tokens` by going through each component of `model` individually

In [ ]:
embeddings = model.pos_embed(tokens) + model.embed(tokens)

In [ ]:
residual = embeddings

In [ ]:
for block in model.blocks:
    residual = block(residual)

In [ ]:
residual = model.ln_final(residual)

In [ ]:
manual_logits = model.unembed(residual)

In [ ]:
torch.allclose(manual_logits, _logits)

True

##### Example 6.3

In [ ]:
type(model)

transformer_lens.HookedTransformer.HookedTransformer

In [ ]:
_embeddings = model.embed(tokens)

In [ ]:
type(model)

transformer_lens.HookedTransformer.HookedTransformer

In [ ]:
_embeddings.shape

torch.Size([8, 15, 768])

Retrieve the text embedding matrix of the `model` and compute the text embedding of `tokens` using it

In [ ]:
W_E = model.W_E

In [ ]:
W_E.shape

torch.Size([50257, 768])

In [ ]:
_manual_embeddings = model.W_E[tokens, :]

In [ ]:
torch.allclose(_embeddings, _manual_embeddings)

True

##### Example 6.4

In [ ]:
len(tokens)

8

In [ ]:
cache["resid_post", -1].shape

torch.Size([8, 15, 768])

In [ ]:
from transformer_lens.utils import get_act_name

In [ ]:
cache

ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.mlp.hook_pre', 'blocks.1.mlp.hook_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_resid_post', 'blocks.2.hook_re

In [ ]:
get_act_name("resid_post", 10)

'blocks.10.hook_resid_post'

##### Example 7

In [ ]:
per_head_residual, labels = cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)

`per_head_residual` is ____

In [ ]:
per_head_residual.shape

torch.Size([144, 8, 768])

Calculate the logit difference. Explaindef residual_stack_to_logit_diff(residual_stack: Float[torch.Tensor, "components batch d_model"], cache: ActivationCache) -> float:
    scaled_residual_stack = cache.apply_ln_to_stack(residual_stack, layer = -1, pos_slice=-1)
    return einsum("... batch d_model, batch d_model -> ...", scaled_residual_stack, logit_diff_directions)/len(prompts)

In [ ]:
answer_residual_directions = model.tokens_to_residual_directions(answer_tokens)

In [ ]:
logit_diff_directions = answer_residual_directions[:, 0] - answer_residual_directions[:, 1]

In [ ]:
per_head_residual, labels = cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)

In [ ]:
def residual_stack_to_logit_diff(residual_stack: Float[torch.Tensor, "components batch d_model"], cache: ActivationCache) -> float:
    scaled_residual_stack = cache.apply_ln_to_stack(residual_stack, layer = -1, pos_slice=-1)
    return einsum("... batch d_model, batch d_model -> ...", scaled_residual_stack, logit_diff_directions)/len(prompts)

In [ ]:
# per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, cache)
# per_head_logit_diffs = einops.rearrange(per_head_logit_diffs, "(layer head_index) -> layer head_index", layer=model.cfg.n_layers, head_index=model.cfg.n_heads)

### Layer Attribution

##### Example 1

#### Decompose Residual StreaM

In [ ]:
x = torch.arange(3*3).view(3, 3)

In [ ]:
from transformer_lens.utils import Slice

In [ ]:
x

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [ ]:
x.shape

torch.Size([3, 3])

Use number

In [ ]:
s1 = Slice(-1)

In [ ]:
x1 = s.apply(x)

In [ ]:
x1

tensor([6, 7, 8])

Use `()`

In [ ]:
s2 = Slice((0, 2))

In [ ]:
x2 = s2.apply(x)

In [ ]:
x2

tensor([[0, 1, 2],
        [3, 4, 5]])

Use `[]`

In [ ]:
s3 = Slice([1, 2])

In [ ]:
x3 = s3.apply(x)

In [ ]:
x3

tensor([[3, 4, 5],
        [6, 7, 8]])

##### Example 2

In [ ]:
per_layer_residual, labels = cache.decompose_resid(layer=3, return_labels=True)

In [ ]:
import torch

In [ ]:
type(cache)

transformer_lens.ActivationCache.ActivationCache

In [ ]:
per_layer_residual.shape

torch.Size([8, 8, 15, 768])

Decompose the residual stream of each layer from `0` to `3` from scratch

In [ ]:
from transformer_lens import utils

In [ ]:
components = []

In [ ]:
text_embeddings = cache["hook_embed"]

In [ ]:
positional_embeddings = cache["hook_pos_embed"]

In [ ]:
components.append(text_embeddings)
components.append(positional_embeddings)

In [ ]:
n_layers = 3

In [ ]:
for i in range(n_layers):
    attn_out_name = utils.get_act_name("attn_out", i)
    mlp_out_name = utils.get_act_name("mlp_out", i)
    components.append(cache[attn_out_name])
    components.append(cache[mlp_out_name])

In [ ]:
components = torch.stack(components, dim=0)

In [ ]:
components.shape

torch.Size([8, 8, 15, 768])

In [ ]:
torch.allclose(per_layer_residual, components)

True

##### Example 3

In [ ]:
len(tokens)

8

In [ ]:
type(cache)

transformer_lens.ActivationCache.ActivationCache

`per_layer_residual` contains the residual stream of each layer

In [ ]:
per_layer_residual.shape

torch.Size([8, 8, 15, 768])

In [ ]:
scaled_per_layer_residual = cache.apply_ln_to_stack(per_layer_residual)

In [ ]:
scaled_per_layer_residual.shape

torch.Size([8, 8, 15, 768])

In [ ]:
scaled_per_layer_residual.mean()

tensor(3.7350e-10)

In [ ]:
mean = per_layer_residual.mean(dim=-1, keepdim=True)

In [ ]:
scale = cache["blocks.3.ln1.hook_scale"]

In [ ]:
_scaled_per_layer_residual = (per_layer_residual - mean) / scale  

In [ ]:
torch.allclose(scaled_per_layer_residual, _scaled_per_layer_residual, rtol=1)

False

##### Example 4

In [ ]:
## final_residual_stream = cache["resid_post", -1]

In [ ]:
len(tokens)

8

In [ ]:
final_residual_stream = cache["blocks.11.hook_resid_post"]

In [ ]:
final_residual_stream.shape

torch.Size([8, 15, 768])

In [ ]:
final_token_residual_stream = final_residual_stream[:, -1, :]

In [ ]:
from transformer_lens.utils import get_act_name

In [ ]:
hook_name = get_act_name("resid_post", 11)

In [ ]:
_text = "Persistence is all you need"

In [ ]:
_tokens = model.to_tokens(_text)

In [ ]:
_tokens

tensor([[50256, 30946, 13274,   318,   477,   345,   761]])

In [ ]:
_original_logits, _cache = model.run_with_cache(_tokens)

In [ ]:
hook_name = "blocks.11.hook_resid_post"

In [ ]:
_final_residual_stream = _cache["resid_post", -1]

In [ ]:
_final_residual_stream.shape

torch.Size([1, 7, 768])

In [ ]:
_final_token_residual_stream = _final_residual_stream[:, -1, :]

In [ ]:
_scaled_final_token_residual_stream = _cache.apply_ln_to_stack(
    _final_token_residual_stream,
    layer=-1,
    pos_slice=-1
)

In [ ]:
_scaled_final_token_residual_stream.shape

torch.Size([1, 768])

In [ ]:
unembed_residual_direction = model.tokens_to_residual_directions(_tokens)

In [ ]:
unembed_residual_direction.shape

torch.Size([1, 7, 768])

In [ ]:
unembed_residual_direction = rearrange(unembed_residual_direction, "1 ... -> ...")

In [ ]:
import einops

In [ ]:
_logits = einops.einsum(
    _scaled_final_token_residual_stream,
    unembed_residual_direction,
    "batch d_model, batch d_model ->"
)

In [ ]:
_logits

tensor(23.2501, grad_fn=<ViewBackward0>)

In [ ]:
_original_logits[:, -1, :]

tensor([[14.2103,  9.5496,  5.1497,  ...,  0.0399, -2.9944, 10.9542]],
       grad_fn=<SliceBackward0>)